<a href="https://colab.research.google.com/github/JhonnyLimachi/Sigmoidal/blob/main/6_Valores_Ausentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 4.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---

# Valores Ausentes

Se você está lidando com bases de dados do mundo real, pode ter certeza que estará lidando com dados incompletos ou valores ausentes.

<center><img src="https://images.unsplash.com/photo-1559311648-d46f5d8593d6?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1033&q=80" height="400px"></center>


Muitas vezes esses dados são inseridos por pessoas, manualmente. Há casos em que você já tinha séries históricas de anos, e em determinado momento alguém resolveu adicionar uma nova coluna.

Existem inúmeras técnicas para lidar com esse tipo de problema, e nesta aula vamos falar de duas das principais abordagens.

Antes, vamos importar os dados do Projeto +BIKE.


In [1]:
DATA_PATH = "http://dl.dropboxusercontent.com/s/yyfeoxqw61o3iel/df_rides.csv"

# importar os pacotes necessários
import pandas as pd

# importar o dataset
df = pd.read_csv(DATA_PATH)

# ver as primeiras entradas
df.head()

,user_gender,user_birthdate,user_residence,ride_date,time_start,time_end,station_start,station_end,ride_duration,ride_late
0,M,1971-06-08,NaN,2018-01-01,06:05:18,06:21:33,11 - Rodoviária 2,41 - Instituto de Artes,16.250000,0.0
1,M,1989-02-11,DF,2018-01-01,06:27:01,06:32:17,26 - Ministério da Saude,28 - CNMP - Conselho Nacional do Ministério Pú...,5.266667,0.0
2,M,1968-07-19,NaN,2018-01-01,06:29:33,06:44:57,11 - Rodoviária 2,43 - Biblioteca Central,15.400000,0.0
3,M,1991-12-19,NaN,2018-01-01,06:53:53,06:59:45,10 - Ministério dos Transportes,6 - Rodoviária,5.866667,0.0
4,M,1969-03-03,DF,2018-01-01,06:58:56,17:40:04,15 - Brasil 21,11 - Rodoviária 2,641.133333,1.0


## Identificando os valores ausentes

Já vimos anteriormente como identificar valores do tipo `NaN`. É indispensável identificar a quantidade de valores ausentes e qual a representatividade dessa quantidade frente ao total de entradas.

In [4]:
# ver a quantidade de valores ausentes
df.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [5]:
# ver a porcentagem de valores ausentes
df.isnull().sum() / df.shape[0]

user_gender       0.001378
user_birthdate    0.000003
user_residence    0.626144
ride_date         0.000000
time_start        0.000000
time_end          0.150650
station_start     0.000000
station_end       0.000000
ride_duration     0.254676
ride_late         0.254676
dtype: float64

## Excluir valores ausentes

Está é uma primeira opção quando você está lidando com valores nulos do tipo `NaN` no seu *DataFrame*.

Entretanto, ela tende a não ser ideal, pois por causa do valor de uma única célula, você elimina os dados existentes em outras colunas. Esta opção deve ser considerada no caso da quantidade de dados nulos serem pequenos a ponto de não ter representatividade no dataset

Eu consigo excluir tanto linhas com valores ausentes quanto colunas inteiras. Para que o Pandas saiba se a sua inteção é de eliminar linhas (`axis=0`) ou colunas (`axis=1`), você deve informar dentro do método `dropna()`.

In [6]:
# eliminar todas as entradas onde existam valores ausentes em `user_gender`
df_clean = df.dropna(subset=['user_gender'], axis=0)

# comparar o antes e o depois
print("Antes:\t{}".format(df.shape))
print("Depois:\t{}".format(df_clean.shape))

Antes:	(287322, 10)
Depois:	(286926, 10)


## Preencher valores

Esta normalmente é a melhor opção, pois permite que você mantenha dados existentes em outras células.

Uma pergunta que normalmente surge é: "mas eu substituo o valor ausente por qual valor?". A resposta para essa pergunta é: depende.

Existem técnicas simples como usar valor mais frequente, media e mediana, assim como há técnicas mais avançadas que envolvem até mesmo o uso de modelos de *machine learning* cuja função é dizer qual valor usar nesses campos.

Vou mostrar como você pode usar a mediana para preencher os campos ausentes da coluna `ride_duration`, com o uso da função `fillna()`

In [7]:
# antes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration      73064
ride_late          73064
dtype: int64

In [8]:
# preencher valores ausentes em `ride_duration` com a mediana
rd_median = df_clean.ride_duration.median()
df_clean = df_clean.fillna({"ride_duration": rd_median})

# ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration          0
ride_late          73064
dtype: int64

No caso da coluna `user_gender`, temos uma variável categórica. O ideal aqui é usar o valor mais frequente para o preenchimento dos valores. Valor ver qual aquele que tem maior recorrência e substituir diretamente na coluna.

In [9]:
# copiar novamente
df_clean = df.copy()

# ver valores ausentes
df_clean.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [10]:
# ver o valor mais frequente
df_clean.user_gender.value_counts()

M    212608
F     74318
Name: user_gender, dtype: int64

In [11]:
# preencher os valores ausentes de user_gender com 'M'
df_clean = df_clean.fillna({"user_gender": 'M'})

# ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64